## Webscrape data from bestplaces.com  using beautiful soup   and selinuium

In [140]:
# conda install selenium 
# download chromedriver: https://sites.google.com/a/chromium.org/chromedriver/downloads      

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import os
import numpy as np
from bs4 import BeautifulSoup
import re

In [141]:
# if needed: pip install requests
import requests

url = 'https://www.bestplaces.net/find/zip.aspx?st=Washington&county=53033'

response = requests.get(url)

In [142]:
response.status_code

200

In [143]:
page = response.text

In [144]:
soup = BeautifulSoup(page,"lxml")

In [145]:
#append  all the links to a list
a_links=[]
for tags in  soup.find_all('div', class_="col-md-4"):
    for a in tags.find_all('a'):
        a_links.append(a['href'])

In [146]:
app_link=['https://www.bestplaces.net'+link.replace('.','') for link in a_links]

In [147]:
len(app_link)

85

In [148]:
url=app_link[0]
    

In [149]:
app_link[0]

'https://www.bestplaces.net/zip-code/washington/auburn/98002'

In [150]:
#chromedriver = f"{os.environ['HOME']}/.local/bin/chromedriver" # path to the chromedriver executable
chromedriver = "/Users/hiranya/Downloads/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

driver = webdriver.Chrome(chromedriver)
driver.get(url)

In [151]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
soup.prettify()


'<!DOCTYPE html>\n<html class="fontawesome-i2svg-active fontawesome-i2svg-complete" xmlns="//www.w3.org/1999/xhtml">\n <head>\n  <!-- Google Tag Manager -->\n  <script async="" src="https://sb.scorecardresearch.com/c2/21065183/cs.js" type="text/javascript">\n  </script>\n  <script src="https://pagead2.googlesyndication.com/pagead/osd.js">\n  </script>\n  <script src="https://cdn.ampproject.org/rtv/011810021759000/amp4ads-host-v0.js">\n  </script>\n  <script async="" src="https://p.cpx.to/p/12133/px.js" type="text/javascript">\n  </script>\n  <script src="https://securepubads.g.doubleclick.net/gpt/pubads_impl_rendering_260.js">\n  </script>\n  <script async="" src="https://rules.quantcount.com/rules-p-e0RhSy58eBqgQ.js">\n  </script>\n  <script async="" src="https://www.googletagservices.com/tag/js/gpt.js" type="text/javascript">\n  </script>\n  <script async="" src="https://www.googletagservices.com/tag/js/gpt.js" type="text/javascript">\n  </script>\n  <script async="" src="//c.amazon-

In [152]:
df=pd.DataFrame(columns=['Comm_time','Zipcode','Median_Income','Median_Age','Violent_crime','Property_crime'])

In [153]:
df.head()

,Comm_time,Zipcode,Median_Income,Median_Age,Violent_crime,Property_crime


In [154]:
data=[tag.text for tag in soup.find_all('p',{"style":"font-size:33px;"})]


In [156]:
data

['4.9%', '$44,930', '$312,600', '34.4', '29.0']

In [157]:
float(soup.find_all('p',{"style":"font-size:33px;"})[3].text)

34.4

**Create functions to scrape features**

In [158]:
def Zipcode(soup):
    try:
        Zip=soup.find('p',{"class":"card-title text-center"}).text.split('(')[1][4:9]
        return Zip
    except:
        None

In [159]:
def Median_Income(soup):
    try:
        Mi=int(soup.find_all('p',{"style":"font-size:33px;"})[1].text.replace('$','').replace(',','').replace(',',''))
        return Mi
    except:
        None

In [160]:
def Median_Age(soup):
    try:
        Mi=float(soup.find_all('p',{"style":"font-size:33px;"})[3].text)
        return Mi
    except:
        None

In [161]:
def Comm_time(soup):
    try:
        Ct=float(soup.find_all('p',{"style":"font-size:33px;"})[4].text)
        return Ct
    except:
        None

In [162]:
df.loc[0,'Comm_time']=Comm_time(soup)
df.loc[0,'Median_Income']=Median_Income(soup)
df.loc[0,'Median_Age']=Median_Age(soup)
df.loc[0,'Zipcode']=Zipcode(soup)
df.loc[0,'Violent_crime']=Violent_crime(soup)
df.loc[0,'Property_crime']=prop_crime(soup)

In [163]:
df.head()

,Comm_time,Zipcode,Median_Income,Median_Age,Violent_crime,Property_crime
0,29,98002,44930,34.4,58.1,57.3


In [164]:
driver.find_elements_by_xpath('//a[contains(text(),"Crime")]')[2].get_attribute('href')

'https://www.bestplaces.net/crime/zip-code/washington/auburn/98002'

In [165]:
crime_data_selector=driver.find_elements_by_xpath('//a[contains(text(),"Crime")]')[2].click()


In [166]:
current_url=driver.current_url

In [167]:
soup1 = BeautifulSoup(driver.page_source,"html.parser")

In [168]:
def Violent_crime(soup):
    try:
        Ct=float(soup1.find_all('h5')[0].text.split(' ')[6][:4])
        return Ct
    except:
        None    


In [169]:
def prop_crime(soup):
    try:
        Ct=float(soup1.find_all('h5')[1].text.split(' ')[6][:4])
        return Ct
    except:
        None
    


Function to srape  features from a link and use selinium to drive through links

In [170]:
def bs_scraper(url,sc_df,ind):
    driver = webdriver.Chrome(chromedriver)
    driver.get(url)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    soup.prettify()
    
     #add to data frame   
    sc_df.loc[ind,'Comm_time']=Comm_time(soup)
    sc_df.loc[ind,'Median_Income']=Median_Income(soup)
    sc_df.loc[ind,'Median_Age']=Median_Age(soup)
    sc_df.loc[ind,'Zipcode']=Zipcode(soup)
    url_new=driver.find_elements_by_xpath('//a[contains(text(),"Crime")]')[2].get_attribute('href')
    #crime_data_selector=driver.find_elements_by_xpath('//a[contains(text(),"Crime")]')[2].click()
    driver.get(url_new)
    time.sleep(1)
    current_url=driver.current_url
    soup1 = BeautifulSoup(driver.page_source,"html.parser")
    sc_df.loc[ind,'Violent_crime']=Violent_crime(soup1)
    sc_df.loc[ind,'Property_crime']=prop_crime(soup1)
    
    driver.close()
    
    return sc_df   
        

**Loopthrough all the page links**

In [173]:
i=11
while i<86:
    #initialize url to first link in the list of zipcode urls
    url=app_link[i]
    
    if i==11:
        #initialize  an empty df
        sc_df=pd.DataFrame(columns=['Comm_time','Zipcode','Median_Income','Median_Age','Violent_crime'\
                                    ,'Property_crime'])
        sc_df=bs_scraper(url,sc_df,i)
        
        sc_df.to_csv('inc_data.csv1',index=False)
    else:
        print('Scraping Page',i)
        sc_df=bs_scraper(url,sc_df,i)
        sc_df.append(sc_df, ignore_index=True)
        print(len(sc_df))
        #append every 10 rows to file
        if i%10==0:
            with open('inc_data.csv1', 'a') as f:
                sc_df.to_csv(f, header=False, index=False)
    
    i=i+1

Scraping Page 12
2
Scraping Page 13
3
Scraping Page 14
4
Scraping Page 15
5
Scraping Page 16
6
Scraping Page 17
7
Scraping Page 18
8
Scraping Page 19
9
Scraping Page 20
10
Scraping Page 21
11
Scraping Page 22
12
Scraping Page 23
13
Scraping Page 24
14
Scraping Page 25
15
Scraping Page 26
16
Scraping Page 27
17
Scraping Page 28
18
Scraping Page 29
19
Scraping Page 30
20
Scraping Page 31
21
Scraping Page 32
22
Scraping Page 33
23
Scraping Page 34
24
Scraping Page 35
25
Scraping Page 36
26
Scraping Page 37
27
Scraping Page 38
28
Scraping Page 39
29
Scraping Page 40
30
Scraping Page 41
31
Scraping Page 42
32
Scraping Page 43
33
Scraping Page 44
34
Scraping Page 45
35
Scraping Page 46
36
Scraping Page 47
37
Scraping Page 48
38
Scraping Page 49
39
Scraping Page 50
40
Scraping Page 51
41
Scraping Page 52
42
Scraping Page 53
43
Scraping Page 54
44
Scraping Page 55
45
Scraping Page 56
46
Scraping Page 57
47
Scraping Page 58
48
Scraping Page 59
49
Scraping Page 60
50
Scraping Page 61
51
Scraping

IndexError: list index out of range